## Data Preparation 

We use Beeline benchmark to benchmark the performance of DeepSEM.
The data preparation process are shown in below.
1. Download raw data from https://doi.org/10.5281/zenodo.3378975, which is provided by BEELINE benchmark
2. Use the preoprocess code in https://github.com/Murali-group/Beeline/blob/master/generateExpInputs.py to generate dataset.

We also provide demo data as shown in ../demo_data/GRN_inference/input 

# Run DeepSEM by using following command:
for cell type specific GRN inference task: python main.py --task non_celltype_GRN --data_file demo_data/GRN_inference/input/500_STRING_hESC/data.csv --net_file demo_data/GRN_inference/input/500_STRING_hESC/label.csv --setting new --alpha 100 --beta 1 --n_epoch 90 --save_name out


for cell type non-specific GRN inference task: python main.py --task celltype_GRN --data_file demo_data/GRN_inference/input/500_ChIP-seq_hESC/data.csv --net_file demo_data/GRN_inference/input/500_ChIP-seq_hESC/label.csv --setting new --alpha 0.1 --beta 0.01 --n_epochs 150  --save_name out

In [40]:
# ! python main.py --task non_celltype_GRN --data_file demo_data/GRN_inference/input/500_STRING_hESC/data.csv --net_file demo_data/GRN_inference/input/500_STRING_hESC/label.csv --setting new --alpha 100 --beta 1 --n_epoch 90 --save_name out
# !python main.py --task celltype_GRN --data_file demo_data/GRN_inference/input/500_ChIP-seq_hESC/data.csv --net_file demo_data/GRN_inference/input/500_ChIP-seq_hESC/label.csv --setting new --alpha 0.1 --beta 0.01 --n_epochs 150  --save_name out
!D:/anaconda/envs/Deep/python.exe C:/Users/lux-c/source/repos/CSB/DeepSEM/main.py --task non_celltype_GRN --data_file demo_data/GRN_inference/input/500_STRING_hESC/data.csv --net_file demo_data/GRN_inference/input/500_STRING_hESC/label.csv --setting new --alpha 100 --beta 1 --n_epoch 90 --save_name out

dir exist
epoch: 1 Ep: 119 Epr: 1.1622069888454667 loss: 3.4098974665006003 mse_loss: 0.9681398322184881 kl_loss: 2.3224788143609962 sparse_loss: 0.11927878608306249
epoch: 2 Ep: 109 Epr: 1.0645425360013099 loss: 3.4068779945373535 mse_loss: 0.9689194609721502 kl_loss: 2.3220602235135934 sparse_loss: 0.11589830492933591
epoch: 4 Ep: 137 Epr: 1.338003003964949 loss: 3.364781141281128 mse_loss: 0.8703611195087433 kl_loss: 2.3772764212141433 sparse_loss: 0.11714358689884345
epoch: 5 Ep: 134 Epr: 1.308703668111702 loss: 3.3627893726030984 mse_loss: 0.8722885151704153 kl_loss: 2.3753663760920367 sparse_loss: 0.11513446519772212
epoch: 7 Ep: 151 Epr: 1.4747332379467686 loss: 3.312417507171631 mse_loss: 0.7266847143570582 kl_loss: 2.4681162064274154 sparse_loss: 0.11761657583216827
epoch: 8 Ep: 176 Epr: 1.7188943700571608 loss: 3.306540826956431 mse_loss: 0.7262569119532903 kl_loss: 2.4632573164999485 sparse_loss: 0.11702662582198779
epoch: 10 Ep: 197 Epr: 1.9239897210298902 loss: 3.290110826

# Calculate EPR values

In [45]:
import pandas as pd
from sklearn.metrics import average_precision_score
import numpy as np
import pandas as pd
output = pd.read_csv('C:/Users/lux-c/source/repos/CSB/DeepSEM/out/GRN_inference_result.tsv',sep='\t')
output['EdgeWeight'] = abs(output['EdgeWeight'])
output = output.sort_values('EdgeWeight',ascending=False)
label = pd.read_csv('C:/Users/lux-c/source/repos/CSB/DeepSEM/demo_data/GRN_inference/input/500_STRING_hESC/label.csv')
TFs = set(label['Gene1'])
Genes = set(label['Gene1'])| set(label['Gene2'])
output = output[output['TF'].apply(lambda x: x in TFs)]
output = output[output['Target'].apply(lambda x: x in Genes)]
label_set = set(label['Gene1']+'|'+label['Gene2'])
output= output.iloc[:len(label_set)]
len(set(output['TF']+'|' +output['Target']) & label_set) / (len(label_set)**2/(len(TFs)*len(Genes)-len(TFs)))



3.8772787779130273

# Calculate AUPR ratio values

In [46]:
def scores():
    !D:/anaconda/envs/Deep/python.exe C:/Users/lux-c/source/repos/CSB/DeepSEM/main.py --task non_celltype_GRN --data_file demo_data/GRN_inference/input/500_STRING_hESC/data.csv --net_file demo_data/GRN_inference/input/500_STRING_hESC/label.csv --setting new --alpha 100 --beta 1 --n_epoch 90 --save_name out
    output = pd.read_csv('C:/Users/lux-c/source/repos/CSB/DeepSEM/out/GRN_inference_result.tsv',sep='\t')
    output.columns.values[0]= "Gene1"
    output.columns.values[1]= "Gene2"
    output['EdgeWeight'] = abs(output['EdgeWeight'])
    output = output.sort_values('EdgeWeight',ascending=False)
    label = pd.read_csv('C:/Users/lux-c/source/repos/CSB/DeepSEM/demo_data/GRN_inference/input//500_STRING_hESC/label.csv')
    TFs = set(label['Gene1'])
    Genes = set(label['Gene1'])| set(label['Gene2'])
    output = output[output['Gene1'].apply(lambda x: x in TFs)]
    output = output[output['Gene2'].apply(lambda x: x in Genes)]
    label_set = set(label['Gene1']+label['Gene2'])
    preds,labels,randoms = [] ,[],[]
    res_d = {}
    l = []
    p= []
    for item in (output.to_dict('records')):
            res_d[item['Gene1']+item['Gene2']] = item['EdgeWeight']
    for item in (set(label['Gene1'])):
            for item2 in  set(label['Gene1'])| set(label['Gene2']):
                if item+item2 in label_set:
                    l.append(1)
                else:
                    l.append(0)
                if item+ item2 in res_d:
                    p.append(res_d[item+item2])
                else:
                    p.append(-1)
    return average_precision_score(l,p)/np.mean(l)
    

In [48]:
for i in range(0,100):

    x=scores()
    with open('C:/Users/lux-c/source/repos/CSB/DeepSEM/test.txt', 'a') as f:
        f.write(str(x))
        f.write('\n')
        f.close()

dir exist
epoch: 1 Ep: 265 Epr: 2.588108000370157 loss: 3.328510562578837 mse_loss: 0.7265408088763555 kl_loss: 2.4711308293044567 sparse_loss: 0.13083896103004614
epoch: 2 Ep: 294 Epr: 2.871334913618212 loss: 3.317398409048716 mse_loss: 0.7235120087862015 kl_loss: 2.4623732306063175 sparse_loss: 0.13151313240329424
epoch: 4 Ep: 315 Epr: 3.076430264590941 loss: 3.274840553601583 mse_loss: 0.5854050119717916 kl_loss: 2.5526435747742653 sparse_loss: 0.1367919941743215
epoch: 5 Ep: 357 Epr: 3.4866209665364 loss: 3.2660588224728904 mse_loss: 0.5772595057884852 kl_loss: 2.5482485430936017 sparse_loss: 0.14055081084370613
epoch: 7 Ep: 365 Epr: 3.5647525288117254 loss: 3.2475539644559226 mse_loss: 0.529085690776507 kl_loss: 2.576056309044361 sparse_loss: 0.1424119956791401
epoch: 8 Ep: 379 Epr: 3.701482762793545 loss: 3.2431480089823403 mse_loss: 0.5252818291385969 kl_loss: 2.574208843211333 sparse_loss: 0.14365741362174353
epoch: 10 Ep: 382 Epr: 3.730782098646792 loss: 3.2350000739097595 mse